In [1]:
#1 Pegando todas as campanhas e seus links

import requests
import pandas as pd
import re

# API das campanhas
api_url = 'https://dashboard.wikiedu.org/lookups/campaign.json'
response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    campanhas = []

    for program in data['campaigns']:
        slug = program['slug']
        campanhas.append([slug])
    df_camp = pd.DataFrame(campanhas, columns=['Links'])

    # Gerando os links completos de cada campanha
    base_url = "https://dashboard.wikiedu.org/campaigns/"
    df_camp['Links'] = df_camp['Links'].apply(lambda row: f'{base_url}{row}')

# Filtrando programas de antes de 2015
def filtro_ano(name):
    match = re.search(r'\b\d{4}\b', name)
    year = int(match.group()) if match else 0
    return year >= 2015 or not match

# Filtrar linhas com base no ano
df_camp = df_camp[df_camp['Links'].apply(filtro_ano)]

In [2]:
#2 Pegando os links dos cursos de cada campanha

from bs4 import BeautifulSoup
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urljoin

def extrair_links_cursos(url):
    links_cursos = set()

    while url:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            course_elements = soup.find_all('a', class_='course-link')
            for course_element in course_elements:
                link = urljoin(url, course_element['href'])  # Para adicionar a terminação ao link completo
                links_cursos.add(link)

            # Verificando se há outras páginas
            pagination_div = soup.find('div', class_='pagination')
            if pagination_div:
                next_page = pagination_div.find('a', class_='next_page')
                if next_page:
                    url = urljoin(url, next_page['href'])
                else:
                    url = None
            else:
                url = None
        else:
            return None
        
    return list(links_cursos)

links_campanhas = df_camp['Links']

# Tentativa de acelerar a extração
with ThreadPoolExecutor(max_workers=10) as executor:
    curso_links = []
    for link in links_campanhas:
        curso_links.extend(extrair_links_cursos(link))

curso_links = list(set(curso_links))

In [3]:
#3 Extraindo as informações requisitadas

# Separado em 3 funções por serem 3 arquivos JSON diferentes

# Nome da campanha
def buscar_campanha(link):
    api_campanha = f"{link}/campaigns.json"
    response_campanha = requests.get(api_campanha)
    
    if response_campanha.status_code == 200:
        campanha_data = response_campanha.json()
        if 'course' in campanha_data and 'campaigns' in campanha_data['course']:
            campaigns = campanha_data['course']['campaigns']
            if campaigns:
                return {'Campanha': campaigns[0]['title']}  # Retorna o título da primeira campanha
    return None

# Informações dos cursos
def buscar_curso(link):
    api_curso = f"{link}/course.json"
    response_curso = requests.get(api_curso)
    
    if response_curso.status_code == 200:
        curso_data = response_curso.json()['course']
        return {
            'Curso': curso_data['title'],
            'Descrição': curso_data['description'],
            'Início': curso_data['start'],
            'Encerramento': curso_data['end'],
            'Instituição': curso_data['school'],
            'Link': link
        }
    
    return None

def buscar_usuarios(link):
    api_users = f"{link}/users.json"
    response_users = requests.get(api_users)
    
    if response_users.status_code == 200:
        users_data = response_users.json()
        instructors = []
        wiki_ed_staff = []
        
        for user in users_data['course']['users']:
            if 'real_name' in user:  # Verifique se 'real_name' está presente
                if user['role'] == 1:
                    if user['real_name'] is None or user['real_name'].lower() == "null":
                        instructors.append(user['username'])
                    else:
                        instructors.append(user['real_name'])
                elif user['role'] == 4:
                    if user['real_name'] is None or user['real_name'].lower() == "null":
                        wiki_ed_staff.append(user['username'])
                    else:
                        wiki_ed_staff.append(user['real_name'])
        
        return {'Facilitadores': instructors, 'Wiki Ed Staff': wiki_ed_staff}
    
    return None

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    campanhas = list(executor.map(buscar_campanha, curso_links))
    cursos = list(executor.map(buscar_curso, curso_links))
    usuarios = list(executor.map(buscar_usuarios, curso_links))

# Criando um dicionário para cada curso
info_cursos = []
for link, campanha, curso, usuario in zip(curso_links, campanhas, cursos, usuarios):
    curso_info = {'Link': link}
    if campanha:
        curso_info.update(campanha)
    info_cursos.append(curso_info)
    if curso:
        curso_info.update(curso)
    info_cursos.append(curso_info)
    if usuario:
        curso_info.update(usuario)
    info_cursos.append(curso_info)

df = pd.DataFrame(info_cursos)

# Salvando em csv
df.to_csv('Dados.csv', index=False)

In [4]:
#4 Tratando o df

# Removando caracteres não aceitos pelo excel
df = df.applymap(lambda x: x.replace('\u000b', '') if isinstance(x, str) else x)

# Formatando as datas e ordenando dos mais recentes para mais antigos
df['Encerramento'] = pd.to_datetime(df['Encerramento'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')
df = df[df['Encerramento'].dt.year >= 2015]
df['Início'] = pd.to_datetime(df['Início'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')
df = df.dropna(subset=['Encerramento'])
df = df.sort_values(by='Encerramento', ascending=False)
df['Encerramento'] = df['Encerramento'].dt.strftime('%d/%m/%Y')
df['Início'] = df['Início'].dt.strftime('%d/%m/%Y')

# Corrigindo os nomes
def converter_lista_para_str(valor):
    if isinstance(valor, list):
        return ', '.join(map(str, valor)) 
    return valor

df = df.applymap(converter_lista_para_str)

# Tirando duplicatas
df = df.drop_duplicates(subset=['Link'])

# Reordenando as colunas
ordem = ['Campanha', 'Curso', 'Descrição', 'Instituição','Início', 'Encerramento', 'Facilitadores', 'Wiki Ed Staff', 'Link']
df = df[ordem]

In [5]:
#5 Criando 2 arquivos em excel

df2 = df.head(700)
df.to_excel('Todos os cursos.xlsx', index=False)
df2.to_excel('700 cursos.xlsx', index=False)